# 导包

In [7]:
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

# 加载数据

In [8]:
dataset = load_dataset("csv", data_files="./问答.csv", split="train")
dataset = dataset.filter(lambda x: x["answer"] is not None)
dataset

Dataset({
    features: ['id', 'question', 'answer'],
    num_rows: 27320
})

In [9]:
datasets = dataset.train_test_split(test_size=0.1)
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer'],
        num_rows: 24588
    })
    test: Dataset({
        features: ['id', 'question', 'answer'],
        num_rows: 2732
    })
})

In [10]:
datasets['train'][:2]

{'id': [21351, 11919],
 'question': ['治疗胃溃疡用什么药物最好。  ', '局灶节段硬化型（IGA）肾炎  '],
 'answer': ['胃溃疡可以检查幽门螺杆菌情况，如果是幽门螺杆菌引起的胃溃疡，唾液乳杆菌+植物乳杆菌组合是杀灭幽门螺杆菌的有效方式。',
  '您好，从您母亲的肾穿刺报告结果可以判断，大约40%左右的肾小球处于硬化期，会逐渐完全失去功能；大约60%的处于病变的早期，积极治疗是可以恢复的。现在的检查结果不是很理想，可能是没有得到很好的治疗。 治疗方法：建议选择中西医结合治疗，主要是调节机体免疫平衡，清除免疫复合物，修复肾小球损伤，增强抵抗力。具体方法有中药汤药，中成药胶囊，肠免疫平衡液输液，中药外治，针灸，药浴等多种治疗方法相结合，临床治疗效果相对比较好。另外可以增强抵抗力，减少上呼吸道感染的机会，以减少疾病复发的次数.']}

# 数据集预处理

In [11]:
tokenizer = AutoTokenizer.from_pretrained("./glm-4-9b-chat", trust_remote_code=True)
tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


ChatGLM4Tokenizer(name_or_path='./glm-4-9b-chat', vocab_size=151329, model_max_length=128000, is_fast=False, padding_side='left', truncation_side='right', special_tokens={'eos_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|endoftext|>', '[MASK]', '[gMASK]', '[sMASK]', '<sop>', '<eop>', '<|system|>', '<|user|>', '<|assistant|>', '<|observation|>', '<|begin_of_image|>', '<|end_of_image|>', '<|begin_of_video|>', '<|end_of_video|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151329: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151330: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151331: AddedToken("[gMASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151332: AddedToken("[sMASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151333: Added

In [12]:
def process_func(example):
    MAX_LENGTH = 768
    input_ids, attention_mask, labels = [], [], []
    instruction = example["question"].strip()     # query
    instruction = tokenizer.apply_chat_template([{"role": "user", "content": instruction}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       )      # '[gMASK] <sop> <|user|> \nquery <|assistant|>'
    
    response = tokenizer("\n" + example["answer"], add_special_tokens=False)        # \n response, 缺少eos token
    input_ids = instruction["input_ids"][0].numpy().tolist() + response["input_ids"] + [tokenizer.eos_token_id]
    attention_mask = instruction["attention_mask"][0].numpy().tolist() + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"][0].numpy().tolist()) + response["input_ids"] + [tokenizer.eos_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [13]:
tokenized_ds = datasets['train'].map(process_func, remove_columns=['id', 'question', 'answer'])
tokenized_ts = datasets['test'].map(process_func, remove_columns=['id', 'question', 'answer'])
tokenized_ds

Map:   0%|          | 0/24588 [00:00<?, ? examples/s]

Map:   0%|          | 0/2732 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 24588
})

In [14]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

'[gMASK] <sop> <|user|> \n局灶节段硬化型（IGA）肾炎 <|assistant|> \n您好，从您母亲的肾穿刺报告结果可以判断，大约40%左右的肾小球处于硬化期，会逐渐完全失去功能；大约60%的处于病变的早期，积极治疗是可以恢复的。现在的检查结果不是很理想，可能是没有得到很好的治疗。 治疗方法：建议选择中西医结合治疗，主要是调节机体免疫平衡，清除免疫复合物，修复肾小球损伤，增强抵抗力。具体方法有中药汤药，中成药胶囊，肠免疫平衡液输液，中药外治，针灸，药浴等多种治疗方法相结合，临床治疗效果相对比较好。另外可以增强抵抗力，减少上呼吸道感染的机会，以减少疾病复发的次数. <|endoftext|>'

In [15]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'\n您好，从您母亲的肾穿刺报告结果可以判断，大约40%左右的肾小球处于硬化期，会逐渐完全失去功能；大约60%的处于病变的早期，积极治疗是可以恢复的。现在的检查结果不是很理想，可能是没有得到很好的治疗。 治疗方法：建议选择中西医结合治疗，主要是调节机体免疫平衡，清除免疫复合物，修复肾小球损伤，增强抵抗力。具体方法有中药汤药，中成药胶囊，肠免疫平衡液输液，中药外治，针灸，药浴等多种治疗方法相结合，临床治疗效果相对比较好。另外可以增强抵抗力，减少上呼吸道感染的机会，以减少疾病复发的次数. <|endoftext|>'

# 创建模型

In [16]:
import torch
model = AutoModelForCausalLM.from_pretrained("./glm-4-9b-chat", trust_remote_code=True, low_cpu_mem_usage=True, device_map="auto",
                                             load_in_4bit=True, 
                                             bnb_4bit_quant_type="nf4",
                                             bnb_4bit_compute_dtype=torch.bfloat16)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
# for name, param in model.named_parameters():
 #    print(name)

# Lora 

# step1 配置文件

In [17]:
from peft import LoraConfig, TaskType, get_peft_model, PeftModel

config = LoraConfig(target_modules=["query_key_value"], modules_to_save=["post_attention_layernorm"])
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=None, inference_mode=False, r=8, target_modules={'query_key_value'}, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=['post_attention_layernorm'], init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

# step2 创建模型

In [18]:
model = get_peft_model(model, config)

In [19]:
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='./glm-4-9b-chat', revision=None, task_type=None, inference_mode=False, r=8, target_modules={'query_key_value'}, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=['post_attention_layernorm'], init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [ ]:
# for name, parameter in model.named_parameters():
  #  print(name)

In [20]:
model.print_trainable_parameters()

trainable params: 2,949,120 || all params: 9,402,900,480 || trainable%: 0.0314


In [21]:
model

PeftModel(
  (base_model): LoraModel(
    (model): ChatGLMForConditionalGeneration(
      (transformer): ChatGLMModel(
        (embedding): Embedding(
          (word_embeddings): Embedding(151552, 4096)
        )
        (rotary_pos_emb): RotaryEmbedding()
        (encoder): GLMTransformer(
          (layers): ModuleList(
            (0-39): 40 x GLMBlock(
              (input_layernorm): RMSNorm()
              (self_attention): SelfAttention(
                (query_key_value): lora.Linear4bit(
                  (base_layer): Linear4bit(in_features=4096, out_features=4608, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Identity()
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=8, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=8, out_features=4608, bias=False)
                  )
           

# 配置训练参数

In [22]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    gradient_checkpointing=True,
    logging_steps=100,
    num_train_epochs=10,
    learning_rate=1e-4,
    remove_unused_columns=False,
    save_strategy="epoch"
)

# 创建训练器

In [23]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds.select(range(10000)),
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


# 模型训练

In [ ]:
trainer.train()

Step,Training Loss
100,3.004600
200,2.675600
300,2.689100
400,2.654200
500,2.628500
600,2.667600
700,2.621400
800,2.631700
900,2.571500
1000,2.589900


/root/miniconda3/envs/chatglm/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in ./glm-4-9b-chat - will assume that the vocabulary was not modified.
  warnings.warn(
/root/miniconda3/envs/chatglm/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in ./glm-4-9b-chat - will assume that the vocabulary was not modified.
  warnings.warn(
/root/miniconda3/envs/chatglm/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in ./glm-4-9b-chat - will assume that the vocabulary was not modified.
  warnings.warn(


In [24]:
from safetensors import safe_open

with safe_open("./chatbot/checkpoint-1875/adapter_model.safetensors", framework="pt") as f:
    for key in f.keys():
        if ".0.post_attention_layernorm" in key:
            print(key)
            print(f.get_tensor(key))

base_model.model.transformer.encoder.layers.0.post_attention_layernorm.weight
tensor([0.1963, 0.1953, 0.1963,  ..., 0.1982, 0.1953, 0.1973],
       dtype=torch.bfloat16)


In [26]:
model.eval()
print(model.chat(tokenizer, "治疗胃溃疡用什么药物最好。", history=[])[0])

治疗胃溃疡通常需要综合治疗，包括药物治疗、饮食调整、生活习惯的改善等。以下是一些常用的药物：

1. **质子泵抑制剂（PPIs）**：这类药物可以显著减少胃酸分泌，是治疗胃溃疡的首选药物。常用的PPIs包括奥美拉唑、兰索拉唑、雷贝拉唑等。

2. **H2受体拮抗剂**：这类药物可以减少胃酸分泌，常用的有雷尼替丁、西咪替丁等。

3. **胃黏膜保护剂**：如硫糖铝、磷酸铝等，可以保护胃黏膜，促进溃疡愈合。

4. **抗生素**：如果胃溃疡是由幽门螺杆菌感染引起的，通常需要联合使用抗生素进行治疗，常用的抗生素有克拉霉素、阿莫西林、甲硝唑等。

5. **胃动力药**：如多潘立酮、莫沙必利等，可以促进胃排空，减轻胃部不适。

具体使用哪种药物，需要根据患者的具体情况和医生的诊断来确定。在治疗过程中，患者应遵医嘱，按时按量服药，并在治疗结束后进行必要的复查。同时，注意以下几点：

- 保持良好的饮食习惯，避免辛辣、油腻、过冷过热等刺激性食物。
- 避免吸烟、饮酒等不良生活习惯。
- 保持良好的心态，适当进行体育锻炼。

如果症状持续或加重，应及时就医。
